# Demo - Extendre ChatGPT amb LangChain

In [1]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import VectorDBQA
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.agents import initialize_agent, Tool
from langchain.text_splitter import CharacterTextSplitter
from langchain import GoogleSearchAPIWrapper, PromptTemplate, LLMChain
from langchain.document_loaders import TextLoader, WebBaseLoader

from dotenv import load_dotenv

load_dotenv();

In [2]:
# Global vars
embeddings = OpenAIEmbeddings()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

### Crear i emmagatzemar a disc els "embeddings"


In [3]:
URL_NOTICIA="https://www.diaridegirona.cat/girona/2023/03/06/vilablareix-comunitat-plaques-fotovoltaiques-84140595.html"

# docs = text_splitter.split_documents(WebBaseLoader(URL_NOTICIA).load())

# noticia_vectordb = FAISS.from_documents(docs, embeddings)
# noticia_vectordb.save_local("faiss_data/diaridegirona_website")

### Carregar de disc els "embeddings"

In [4]:
noticia_vectordb = FAISS.load_local("faiss_data/diaridegirona_website", embeddings)

### ChatGPT prompt - Preguntes i respostes sobre l'article del Diari de Girona

In [5]:
noticia = VectorDBQA.from_chain_type(llm=ChatOpenAI(temperature=0.7), chain_type="stuff", vectorstore=noticia_vectordb)

In [6]:
noticia.run("Fes un resum de la notícia en una frase")

'L’Ajuntament de Vilablareix impulsa la creació d’una comunitat local d’energia en el municipi instal·lant plaques solars fotovoltaiques a la coberta del pavelló municipal per subministrar energia elèctrica als edificis municipals i als veïns que vulguin formar part.'

In [7]:
noticia.run("Com es diu la regidora de medi ambient de l'ajuntament de Vilablareix?")

"La regidora de medi ambient de l'ajuntament de Vilablareix es diu Imma Vilà."

In [8]:
noticia.run("Quina és la inversió inicial per a la instal.lació de plaques solars a Vilablareix?")

'La inversió inicial per a la instal·lació de plaques solars a Vilablareix és de 48.878,05 euros per una instal·lació de 58,65 kWp i està subvencionada per la Diputació de Girona amb 36.000,00 euros.'

### Chat amb "memòria" i cerca a Google per complementar respostes

In [9]:
tools = [
    Tool(
        name = "Google Search",
        func=GoogleSearchAPIWrapper().run,
        description="Cerca a Google per complementar informació"
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

agent_chain = initialize_agent(tools, ChatOpenAI(temperature=0.7), agent="conversational-react-description", verbose=True, max_iterations=10, memory=memory)


In [11]:
agent_chain.run(input="Quin es el número de habitants de Vilablareix, Girona? ")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Google Search
Action Input: Population of Vilablareix, Girona
Observation: Vilablareix is a small village in the province of Girona and autonomous community of Catalonia, Spain. Vilablareix. Municipality. that the reported populations may be closely related to plants from the United States. ... The population from Vilablareix (Girona province). The population from Vilablareix (Girona province) grows in a grassland area, formely arable fields, which are grazed by sheeps in summer. Vilablareix (Girona, Cataluña, Spain) with population statistics, charts, map, location, weather and web information. Restaurant Mas Marroch, Vilablareix: See 103 unbiased reviews of Restaurant Mas ... Luckily Girona area is full of great restaurants where they welcome also ... The main objective of the ASTRID 21 FOUNDATION is to ensure that people with Down's syndrome or other intellectual disabilities can take part in socie

'According to available data, the population of Vilablareix, Girona, Spain is approximately 2,194 residents.'

In [12]:
agent_chain.run(input="Assumint una ocupació de 2.5 habitants per llar. Quin porcentatge de habitants implicaria 160 llars?. Contesta en català")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Si assumim una ocupació de 2.5 habitants per llar, el número total de residents seria de 400. Això és perquè 160 llars x 2.5 habitants per llar = 400 habitants. Per tant, el percentatge de la població que representen aquests 160 llars seria del 18.29% (160/2,194 x 100).

> Finished chain.


'Si assumim una ocupació de 2.5 habitants per llar, el número total de residents seria de 400. Això és perquè 160 llars x 2.5 habitants per llar = 400 habitants. Per tant, el percentatge de la població que representen aquests 160 llars seria del 18.29% (160/2,194 x 100).'